In [70]:
import numpy as np
import pandas as pd
import pickle

In [71]:
# Load test set
with open('data/transcripts_test.pickle', 'rb') as f:
    transcripts_test = pickle.load(f)

In [72]:
# Need to refer to the original transcripts because the cleaned data does not include
# stopwords. Improves readability for manually labeling the data.
statements = pd.read_pickle('data/statements_mini')
statements.head()

,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Presentation,NaN
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Operator,"Greetings, and welcome to the Ark Restaurants ..."
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Thank you, operator. Good morning, and thank y..."
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Michael Weinstein,"Hi, everybody. If you read the release, we're ..."
4,2,UniFirst Corp.,NYSE,UNF,2017-01-04 13:33:06,UniFirst's (UNF) CEO Ronald Croatti on Q1 2017...,UniFirst Corporation.,Q1 2017 Earnings Conference Call,NaN,UNF,"January 4, 2017 10:00 AM ET",1,Presentation,NaN


In [73]:
qna = pd.read_pickle('data/qna_mini')
qna.head()

,num,file-company-name,file-exchange,file-symbol,file-published-on,file-title,company-name,quarter-year-event,symbol-exchange,symbol,datetime,order,name,content
0,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",1,Operator,[Operator Instructions] Our first question com...
1,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",2,Bruce Geller,Hi Good morning guys.
2,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",3,Bob Stewart,"Hi Bruce, how are you?"
3,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",4,Bruce Geller,Good thanks. Glad to hear things have turned a...
4,1,Ark Restaurants Corp.,NASDAQ,ARKR,2017-01-01 06:49:04,Ark Restaurants' (ARKR) CEO Michael Weinstein ...,Ark Restaurants Corp.,Q4 2016 Earnings Conference Call,NaN,ARKR,"December 30, 2016 10:00 A.M. ET",5,Bob Stewart,"Yes, that includes an additional week last yea..."


In [74]:
with open('data/id_to_row.pickle', 'rb') as f:
    id_to_row = pickle.load(f)

In [75]:
# Start with 10 transcripts from the test set -> need to compile 50 questions
examples = transcripts_test[len(transcripts_test)-10:len(transcripts_test)]

In [76]:
def extract_full_transcript(cleaned_transcript):
    transcript_id = cleaned_transcript[1]
    statements_row, qna_row = id_to_row[transcript_id]
    statement, counter, num = [], 0, statements.iloc[statements_row]['num']
    while(True):
        if statements.iloc[statements_row+counter]['num'] != num: break
        statement.append(statements.iloc[statements_row+counter]['content'])
        counter+=1
    questions, counter, num = [], 0, qna.iloc[statements_row]['num']
    while(True):
        if qna.iloc[statements_row+counter]['num'] != num: break
        questions.append(qna.iloc[qna_row+counter]['content'])
        counter+=1
    return statement, questions

In [131]:
def create_qa_pairs(cleaned_transcript):
    qa_pairs = []
    QA, index = cleaned_transcript[3], 0
    while(True):
        while index < len(QA) and QA[index][1] == 1: index+=1
        if index >= len(QA): break
        question = QA[index][0]
        while index < len(QA) and QA[index][1] == 0: 
            question += QA[index][0]
            index+=1
        if index >= len(QA): break
        answer = QA[index][0]
        while index < len(QA) and QA[index][1] == 1: 
            question += QA[index][0]
            index+=1
        qa_pairs.append((question, answer))
    return qa_pairs

In [144]:
# In order to hand label examples, we will first read through the given transcript. 
# After reading through the given transcripts, we will examine each of the questions, and
# determine which chunk of the transcript is most relevant to the question. 
# USE 2 (11), 3(19), 5(14), and 7(16)
cleaned_transcript = examples[2]
qa_pairs = create_qa_pairs(examples[2])
readable_transcript = extract_full_transcript(examples[2])
print(len(qa_pairs))

12


In [136]:
print(readable_transcript[0])

[nan, "Good morning. I'm Bill Robertson, Vice President of Finance for Visteon. Welcome to our Earnings Call for the Fourth Quarter and Full Year of 2016. Please note this call is being recorded and all lines have been placed on listen-only mode to prevent background noise.; ; Before we begin this morning's call, I would like to remind you this presentation contains forward-looking statements within the meaning of the Private Securities Litigation Reform Act of 1995. Forward-looking statements are not guarantees of future results and conditions or rather are subject to various factors, risks, and uncertainties that could cause our actual results to differ materially from those expressed in these statements. Please refer to the Slide entitled forward-looking information for further details.; ; Presentation materials for today's call were posted on the Investors Section of Visteon's website this morning. Please visit www.visteon.com/earnings to download the material if you've not already

In [146]:
print(readable_transcript[1])

['[Operator Instructions].; ; Your first question comes from the line of David Leiker with Baird. Please go ahead.', 'Hi good morning, this is Joe Vruwink for David.', 'Good morning, Joe.', "I wanted to talk about new business awards, if I just look at the quarterly cadence of what you have been winning, when you have been winning it seems that Q2 of last year suddenly something happens where Visteon is booking at a much higher level than it had been and subsequent quarters you continue to book at this elevated level. Now I don't know if it's easy to say CES 2016 was a very important event, a lot of good stuff shown, you've talked about meeting with more automakers and so I can tie up back into that. But the reason I ask is CES 2017, so what you did last month seemed to be an even bigger and better event, you certainly had a lot of new products, Phoenix's going to be meaningful and so is that event the driver where we can expect another stair step in new business awards may be next qua

In [152]:
print("The question:")
print(qa_pairs[1][0])
print("")
print("The answer:")
print(qa_pairs[1][1])
print("The model output:")


wanted talk new business awards look quarterly cadence winning winning seems q2 last year suddenly something happens visteon booking much higher level subsequent quarters continue book elevated level know easy say ces 2016 important event lot good stuff shown talked meeting automakers tie back reason ask ces 2017 last month seemed even bigger better event certainly lot new products phoenix going meaningful event driver expect another stair step new business awards may next quarter quarter thereafter sustain higher levelwanted talk new business awards look quarterly cadence winning winning seems q2 last year suddenly something happens visteon booking much higher level subsequent quarters continue book elevated level know easy say ces 2016 important event lot good stuff shown talked meeting automakers tie back reason ask ces 2017 last month seemed even bigger better event certainly lot new products phoenix going meaningful event driver expect another stair step new business awards may ne

In [139]:
print(cleaned_transcript[2])

['good morning bill robertson vice president finance visteon welcome earnings call fourth quarter full year 2016 please note call recorded lines placed mode prevent background noise begin morning call would like remind presentation contains statements within meaning private securities litigation reform act 1995 statements guarantees future results conditions rather subject various factors risks uncertainties could cause actual results differ materially expressed statements please refer', 'slide entitled information details presentation materials today call posted investors section visteon website morning please visit download material already done form filed earlier morning news release joining us today sachin lawande president chief executive officer christian garcia executive vice president chief financial officer scheduled call one hour open lines questions sachin christian remarks please limit questions one question one thank joining us call today', 'turn sachin', 'thank bill page 

In [140]:
cleaned_transcript[2][23]

'position cockpit hmi extending capability exciting transformative area automotive electronics autonomous driving technology moving page 11 page next would like focus business environment facing 2017 key priorities visteon first expecting modest improvement global production volume 2017 versus 2016 ihs expecting global production volumes grow 2017 china europe showing positive growth north america forecast reduce moreover top 10 visteon customers forecast grow less 1 weighted'

In [ ]:
[(None), (20, 21, 22), ]